In [1]:
from kafka import KafkaProducer
import time
import numpy as np
import pandas as pd

In [2]:
#Takes about one minute to load
data=pd.read_csv("../../data/data.conv.txt.gz",header=None,sep=" ")
data.columns=["Date","Hour","Sensor","Value","Voltage"]
data=data.sort_values(['Date','Hour']).reset_index(drop=True)

data['datetime']=pd.to_datetime(data.Date+' '+data.Hour)
data['relative_datetime']=data['datetime']-data['datetime'][0]
data['seconds']=data['relative_datetime'].dt.total_seconds()

sensorId_type=data.Sensor.str.split("-",expand=True)
sensorId_type.columns=['SensorId','Type']
data['SensorId']=sensorId_type['SensorId'].astype(int)
data['Type']=sensorId_type['Type'].astype(int)

In [3]:
#Drop features not needed for the simulation
data=data.drop(['datetime','relative_datetime','Sensor','Date','Hour','Voltage'],axis=1)

In [4]:
data[:5]

,Value,seconds,SensorId,Type
0,17.6364,0.000,41,0
1,16.6956,0.007,44,0
2,45.7037,0.092,41,1
3,2.3000,0.237,44,2
4,47.9942,0.285,44,1


In [5]:
temp=data[data.Type==0]
temp=temp.reset_index(drop=True)

In [6]:
day_to_predict = 1 # must be at least 1
temp_train = temp[temp.seconds < day_to_predict*86400]
temp_test = temp[(temp.seconds >= day_to_predict*86400) & (temp.seconds < (day_to_predict+1)*86400)]

In [7]:
temp_sample = temp_train[:20]
print(temp_sample.iloc[0])

Value       17.6364
seconds      0.0000
SensorId    41.0000
Type         0.0000
Name: 0, dtype: float64


In [8]:
producer = KafkaProducer(bootstrap_servers='kafka1:19092,kafka2:29092,kafka3:39092')

In [9]:
# Sensors to predict
sensors_to_predict = [1, 24]
temp_train_y = temp_train[temp_train.SensorId.isin(sensors_to_predict)][:10]
print(temp_train_y[:10])

       Value  seconds  SensorId  Type
64   18.1362   53.492        24     0
79   19.2436   79.124         1     0
81   18.1166   79.230        24     0
114  18.1264  108.864        24     0
146  18.1362  138.638        24     0
188  19.2240  169.155         1     0
224  19.2142  200.931         1     0
247  18.1460  229.040        24     0
305  19.1848  264.276         1     0
321  18.1264  290.305        24     0


In [ ]:
import json

time_resolution = 60 # window of time we use to extract other time-related temperatures
interval = 1 # waiting interval between the sending of each temperature to predict
n = 10 # short-time memory length (in time_resolution)
m = 1 # long-time memory length (in days, i.e. 86400 s)
for iy, ry in temp_train_y.iterrows():
    tic = time.time()
    y = ry.Value
    id_y = ry.SensorId
    bin_y = ry.seconds//time_resolution
    day_y = ry.seconds//86400
    temp_past = temp_train_y[((temp_train_y.seconds//time_resolution < bin_y)
                             & (temp_train_y.seconds//time_resolution >= bin_y - n))
                             | (temp_train_y.seconds//86400 < )
                             & (temp_train.SensorId == id_y)]